In [14]:
import pandas as pd
import numpy as np
from sodapy import Socrata
import collections
import re
from time import time
import math
import feather
from collections import defaultdict
import geopandas
from datetime import datetime

In [3]:
# Load SF Police Data
df = feather.read_dataframe('data/sf_incidents.feather')

In [ ]:
# Load SF Census Track Data
census_tracks = geopandas.read_file('data/census2010_ sf_tracks.geojson')
census_tracks = census_tracks[['geometry', 'name']] <-- Keep only census track name / geometrydf

# Load SF Census Block Data
census_blocks = geopandas.read_file('data/census2010_ sf_blocks.geojson')
census_blocks = census_blocks[['geometry', 'name']] <-- Keep only census track name / geometrydf

In [7]:
print(df.describe())
print(df.head())
print(census_tracks.head())
print(census_blocks.head())

       incident_category        incident_date          incident_description  \
count             701915               701932                        701932   
unique                89                 1566                          1547   
top        LARCENY/THEFT  2018-04-13 00:00:00  GRAND THEFT FROM LOCKED AUTO   
freq              146464                  942                         66440   
first                NaN  2015-01-01 00:00:00                           NaN   
last                 NaN  2019-04-15 00:00:00                           NaN   

       incident_dow incident_number incident_time             latitude  \
count        701932          701932        701932               691557   
unique            7          427206          1440                47523   
top          Friday       180187811         12:00  -122.40340479147905   
freq         107685              27         17712                 9246   
first           NaN             NaN           NaN                  NaN   
la

In [6]:
# spatial join
df_wcensus = geopandas.sjoin(df, census_tracks, how="inner", op='within')
df_wcensus = geopandas.sjoin(df_wcensus, census_blocks, how="inner", op='within')

,incident_category,incident_date,incident_description,incident_dow,incident_number,incident_time,latitude,longitude,police_district
0,Homicide,2018-08-20,"Homicide, W/ Other Weapon",Monday,180624631,07:45,37.78610103879033,-122.41808809883916,Northern
1,Assault,2018-08-11,"Assault, Aggravated, W/ Other Weapon",Saturday,180602295,19:10,37.75683373380551,-122.40669900268833,Mission
2,Other Miscellaneous,2018-08-14,"Resisting, Delaying, or Obstructing Peace Off...",Tuesday,189002400,14:15,37.770555975146884,-122.39115601762397,Southern
3,Larceny Theft,2018-08-14,"Theft, Bicycle, >$950",Tuesday,180609984,15:07,37.766691459464916,-122.3926618696727,Southern
4,Other Miscellaneous,2018-08-14,"Burglary Tools, Possession Of",Tuesday,180609984,15:07,37.766691459464916,-122.3926618696727,Southern


In [20]:
# Weather Data via NOAA request
weather = pd.read_csv('data/weather.csv')
weather = weather[['NAME',  'LATITUDE',  'LONGITUDE', 'DATE', 'PRCP', 'TAVG']]
print(weather.describe())
print(weather.head())

            LATITUDE      LONGITUDE           PRCP          TAVG
count  120530.000000  120530.000000  105010.000000  20127.000000
mean       37.804167    -122.286561       0.097325     58.090277
std         0.207808       0.221397       0.319468      9.046352
min        37.375400    -122.702200       0.000000     33.000000
25%        37.619700    -122.482500            NaN           NaN
50%        37.869800    -122.260500            NaN           NaN
75%        37.983300    -122.115000            NaN           NaN
max        38.210200    -121.865500       7.570000    109.000000
                      NAME  LATITUDE  LONGITUDE        DATE  PRCP  TAVG
0  BERKELEY 0.9 SSE, CA US   37.8552  -122.2892  2017-07-01   0.0   NaN
1  BERKELEY 0.9 SSE, CA US   37.8552  -122.2892  2017-07-02   0.0   NaN
2  BERKELEY 0.9 SSE, CA US   37.8552  -122.2892  2017-07-03   0.0   NaN
3  BERKELEY 0.9 SSE, CA US   37.8552  -122.2892  2017-07-04   0.0   NaN
4  BERKELEY 0.9 SSE, CA US   37.8552  -122.2892  2017-0

In [ ]:
# Merge Weather with data via Day 
# Data, match site of detector with census track <- find closest
# Attach detectors PRCP and TAVG (temperature average)

In [ ]:
# Create time-period units



In [ ]:
# Collapse By Census_Track

In [ ]:
# Collapse BY Time Period 